<H4> In this Model I Made a Content Based Recommender Model using Vectorization, Cosine Similarity and optimize it futher using weight average rating that I used in Simple Recommender System </H4>

<H4> Now Here I Used the dumped data from simple recommender system file </H4> 

In [266]:
# First I imported the necessary Libraries

# for basic Operations
import pandas as pd
import numpy as np

# for data analysis
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import ast
from ast import literal_eval

# for vectorization and Cosine Distance or similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# for stemming the words
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

# for avoiding warnings
import warnings; warnings.simplefilter('ignore')

# for dumping the dataset and use it in other IDE
import pickle

# for Interactive Analysis(@interact)
import ipywidgets as widgets
from ipywidgets import *
from ipywidgets import interact

In [267]:
# Reading the movie dataset 
movies = pd. read_csv('movies_data.csv')


# Checking the dataset
movies.head(3)

,Unnamed: 0,genres,id,keywords,original_title,overview,popularity,release_date,tagline,vote_average,vote_count,cast,crew,weigh_avg_rating
0,1881,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...",278,"[{""id"": 378, ""name"": ""prison""}, {""id"": 417, ""n...",The Shawshank Redemption,Framed in the 1940s for the double murder of h...,136.747729,1994-09-23,Fear can hold you prisoner. Hope can set you f...,8.5,8205,"[{""cast_id"": 3, ""character"": ""Andy Dufresne"", ...","[{""credit_id"": ""52fe4231c3a36847f800b153"", ""de...",8.038342
1,662,"[{""id"": 18, ""name"": ""Drama""}]",550,"[{""id"": 825, ""name"": ""support group""}, {""id"": ...",Fight Club,A ticking-time-bomb insomniac and a slippery s...,146.757391,1999-10-15,Mischief. Mayhem. Soap.,8.3,9413,"[{""cast_id"": 4, ""character"": ""The Narrator"", ""...","[{""credit_id"": ""55731b8192514111610027d7"", ""de...",7.921436
2,65,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...",155,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight,Batman raises the stakes in his war on crime. ...,187.322927,2008-07-16,Why So Serious?,8.2,12002,"[{""cast_id"": 35, ""character"": ""Bruce Wayne"", ""...","[{""credit_id"": ""55a0eb4a925141296b0010f8"", ""de...",7.905627


In [268]:
# Checking the dimension of movies dataset
movies.shape

(4505, 14)

# PreProcessing of Data

In [269]:
# here I rename the column original_title to title as they both are same
movies = movies.rename(columns={'original_title':'title'})

In [270]:
movies['title'].duplicated().sum()

2

In [271]:
# removing the duplicates

movies = movies.drop_duplicates(subset=["title"], keep='first')
movies.shape

(4503, 14)

In [272]:
# Making a final data by dropping unnecessary columns

final_data = movies[['id', 'title', 'overview','genres', 'release_date', 'keywords', 'cast', 'crew', 'weigh_avg_rating']]

In [273]:
final_data.head(3)

,id,title,overview,genres,release_date,keywords,cast,crew,weigh_avg_rating
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...",1994-09-23,"[{""id"": 378, ""name"": ""prison""}, {""id"": 417, ""n...","[{""cast_id"": 3, ""character"": ""Andy Dufresne"", ...","[{""credit_id"": ""52fe4231c3a36847f800b153"", ""de...",8.038342
1,550,Fight Club,A ticking-time-bomb insomniac and a slippery s...,"[{""id"": 18, ""name"": ""Drama""}]",1999-10-15,"[{""id"": 825, ""name"": ""support group""}, {""id"": ...","[{""cast_id"": 4, ""character"": ""The Narrator"", ""...","[{""credit_id"": ""55731b8192514111610027d7"", ""de...",7.921436
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...",2008-07-16,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 35, ""character"": ""Bruce Wayne"", ""...","[{""credit_id"": ""55a0eb4a925141296b0010f8"", ""de...",7.905627


In [274]:
# checking for the missing values
final_data.isnull().sum()

id                  0
title               0
overview            1
genres              0
release_date        1
keywords            0
cast                0
crew                0
weigh_avg_rating    0
dtype: int64

In [275]:
# removing the missing values rows
final_data.dropna(inplace = True)
final_data.isnull().sum()

id                  0
title               0
overview            0
genres              0
release_date        0
keywords            0
cast                0
crew                0
weigh_avg_rating    0
dtype: int64

In [276]:
# Now here I checked for duplicate rows

# lets print the no. of rows before removing Duplicates
print("No. of Rows Before Removing Duplicates: ",movies.shape[0])

# Removing all the duplicates from the data
movies.drop_duplicates(subset = None, keep = 'first', inplace = True)

# lets print the no. of rows after removing Duplicates
print("No. of Rows After Removing Duplicates: ",movies.shape[0])

No. of Rows Before Removing Duplicates:  4503
No. of Rows After Removing Duplicates:  4503


In [277]:
# fetching release year from the release date and making a column for it

final_data['year'] = pd.to_datetime(final_data['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

# now removing the release_date column
final_data = final_data.drop(['release_date'], axis = 1) 

# now checking the final movie dataset
final_data.head()

,id,title,overview,genres,keywords,cast,crew,weigh_avg_rating,year
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...","[{""id"": 378, ""name"": ""prison""}, {""id"": 417, ""n...","[{""cast_id"": 3, ""character"": ""Andy Dufresne"", ...","[{""credit_id"": ""52fe4231c3a36847f800b153"", ""de...",8.038342,1994
1,550,Fight Club,A ticking-time-bomb insomniac and a slippery s...,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 825, ""name"": ""support group""}, {""id"": ...","[{""cast_id"": 4, ""character"": ""The Narrator"", ""...","[{""credit_id"": ""55731b8192514111610027d7"", ""de...",7.921436,1999
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 35, ""character"": ""Bruce Wayne"", ""...","[{""credit_id"": ""55a0eb4a925141296b0010f8"", ""de...",7.905627,2008
3,680,Pulp Fiction,"A burger-loving hit man, his philosophical par...","[{""id"": 53, ""name"": ""Thriller""}, {""id"": 80, ""n...","[{""id"": 396, ""name"": ""transporter""}, {""id"": 14...","[{""cast_id"": 2, ""character"": ""Vincent Vega"", ""...","[{""credit_id"": ""52fe4269c3a36847f801caa3"", ""de...",7.885403,1994
4,27205,Inception,"Cobb, a skilled thief who commits corporate es...","[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...","[{""id"": 1014, ""name"": ""loss of lover""}, {""id"":...","[{""cast_id"": 1, ""character"": ""Dom Cobb"", ""cred...","[{""credit_id"": ""56e8462cc3a368408400354c"", ""de...",7.850750,2010


In [278]:
# now here I change the genres column format from dictionary to List using literal_eval

# the dataset contain genres in the form of id and name. The genre of movie is present in the name section 

# so I did this
final_data['genres'] = final_data['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

final_data.head(3)

,id,title,overview,genres,keywords,cast,crew,weigh_avg_rating,year
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[Drama, Crime]","[{""id"": 378, ""name"": ""prison""}, {""id"": 417, ""n...","[{""cast_id"": 3, ""character"": ""Andy Dufresne"", ...","[{""credit_id"": ""52fe4231c3a36847f800b153"", ""de...",8.038342,1994
1,550,Fight Club,A ticking-time-bomb insomniac and a slippery s...,[Drama],"[{""id"": 825, ""name"": ""support group""}, {""id"": ...","[{""cast_id"": 4, ""character"": ""The Narrator"", ""...","[{""credit_id"": ""55731b8192514111610027d7"", ""de...",7.921436,1999
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[Drama, Action, Crime, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 35, ""character"": ""Bruce Wayne"", ""...","[{""credit_id"": ""55a0eb4a925141296b0010f8"", ""de...",7.905627,2008


In [279]:
# same Thing that I done above do with the Keywords

final_data['keywords'] = final_data['keywords'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

final_data.head(2)

,id,title,overview,genres,keywords,cast,crew,weigh_avg_rating,year
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[Drama, Crime]","[prison, corruption, police brutality, prison ...","[{""cast_id"": 3, ""character"": ""Andy Dufresne"", ...","[{""credit_id"": ""52fe4231c3a36847f800b153"", ""de...",8.038342,1994
1,550,Fight Club,A ticking-time-bomb insomniac and a slippery s...,[Drama],"[support group, dual identity, nihilism, rage ...","[{""cast_id"": 4, ""character"": ""The Narrator"", ""...","[{""credit_id"": ""55731b8192514111610027d7"", ""de...",7.921436,1999


In [280]:
# here I am taking only Top 3 cast from the cast columns as many people watch movies on the basis of actors/actresses

def Leading_cast(text):
    List = []
    j = 0
    for i in ast.literal_eval(text):
        if j < 3:
            List.append(i['name'])
        j+=1
    return List 

In [281]:
# fetching the Top 3 leading cast
final_data['cast'] = final_data['cast'].apply(Leading_cast)

In [282]:
# checking the dataset
final_data.head(2)

,id,title,overview,genres,keywords,cast,crew,weigh_avg_rating,year
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[Drama, Crime]","[prison, corruption, police brutality, prison ...","[Tim Robbins, Morgan Freeman, Bob Gunton]","[{""credit_id"": ""52fe4231c3a36847f800b153"", ""de...",8.038342,1994
1,550,Fight Club,A ticking-time-bomb insomniac and a slippery s...,[Drama],"[support group, dual identity, nihilism, rage ...","[Edward Norton, Brad Pitt, Meat Loaf]","[{""credit_id"": ""55731b8192514111610027d7"", ""de...",7.921436,1999


In [283]:
# Now similiary there may me many movies whose directors are same. so it will make easy to find similarity between movies
# according to directors
def get_director(crew):
    L = []
    for i in ast.literal_eval(crew):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [284]:
# fetching the director
final_data['crew'] =  final_data['crew'].apply(get_director)

In [285]:
# renaming crew to director
final_data = final_data.rename(columns={'crew':'director'})

In [286]:
# For use as dump data in other id
New_data = final_data

# now i dumped the New_data to use it in my IDe To build Web app
pickle.dump(New_data,open('New_data.pkl','wb'))

In [287]:
New_data.shape

(4501, 9)

In [288]:
# Now Here I changing the Upper case strings inside the list cast to lower case and removing space between the strings
# as it will be easy to compare.  

final_data['cast'] = final_data['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [289]:
final_data.head(2)

,id,title,overview,genres,keywords,cast,director,weigh_avg_rating,year
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[Drama, Crime]","[prison, corruption, police brutality, prison ...","[timrobbins, morganfreeman, bobgunton]",[Frank Darabont],8.038342,1994
1,550,Fight Club,A ticking-time-bomb insomniac and a slippery s...,[Drama],"[support group, dual identity, nihilism, rage ...","[edwardnorton, bradpitt, meatloaf]",[David Fincher],7.921436,1999


In [290]:
# Same thing doing with the director, keywords, genres columns 

final_data['director'] = final_data['director'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [291]:
final_data['keywords'] = final_data['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [292]:
final_data['genres'] = final_data['genres'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [293]:

# Now checking the final-data

final_data.head(3)

,id,title,overview,genres,keywords,cast,director,weigh_avg_rating,year
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[drama, crime]","[prison, corruption, policebrutality, prisonce...","[timrobbins, morganfreeman, bobgunton]",[frankdarabont],8.038342,1994
1,550,Fight Club,A ticking-time-bomb insomniac and a slippery s...,[drama],"[supportgroup, dualidentity, nihilism, rageand...","[edwardnorton, bradpitt, meatloaf]",[davidfincher],7.921436,1999
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[drama, action, crime, thriller]","[dccomics, crimefighter, secretidentity, scare...","[christianbale, heathledger, aaroneckhart]",[christophernolan],7.905627,2008


In [294]:
# stemming for keywords as all other columns that we are going to include may not have words like boy and boys
stemmer = SnowballStemmer('english')

# example
stemmer.stem('Boys')

'boy'

In [295]:
final_data['keywords'] = final_data['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

In [296]:
final_data.head(3)

,id,title,overview,genres,keywords,cast,director,weigh_avg_rating,year
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[drama, crime]","[prison, corrupt, policebrut, prisoncel, delin...","[timrobbins, morganfreeman, bobgunton]",[frankdarabont],8.038342,1994
1,550,Fight Club,A ticking-time-bomb insomniac and a slippery s...,[drama],"[supportgroup, dualident, nihil, rageandh, ins...","[edwardnorton, bradpitt, meatloaf]",[davidfincher],7.921436,1999
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[drama, action, crime, thriller]","[dccomic, crimefight, secretident, scarecrow, ...","[christianbale, heathledger, aaroneckhart]",[christophernolan],7.905627,2008


In [297]:

# final details which is going to be use for checking similarity

final_data['details'] = final_data['genres'] + final_data['cast'] + final_data['keywords'] + final_data['director']

In [298]:
final_data.head(3)

,id,title,overview,genres,keywords,cast,director,weigh_avg_rating,year,details
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[drama, crime]","[prison, corrupt, policebrut, prisoncel, delin...","[timrobbins, morganfreeman, bobgunton]",[frankdarabont],8.038342,1994,"[drama, crime, timrobbins, morganfreeman, bobg..."
1,550,Fight Club,A ticking-time-bomb insomniac and a slippery s...,[drama],"[supportgroup, dualident, nihil, rageandh, ins...","[edwardnorton, bradpitt, meatloaf]",[davidfincher],7.921436,1999,"[drama, edwardnorton, bradpitt, meatloaf, supp..."
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[drama, action, crime, thriller]","[dccomic, crimefight, secretident, scarecrow, ...","[christianbale, heathledger, aaroneckhart]",[christophernolan],7.905627,2008,"[drama, action, crime, thriller, christianbale..."


In [299]:

# removing the columns keywords', 'director', 'overview', 'cast', 'genres','year' as there work is done

final_data = final_data.drop(columns = ['keywords', 'director', 'overview', 'cast', 'genres','year'])

In [300]:

# coverting our final details column to string for comparison

final_data['details'] = final_data['details'].apply(lambda x: " ".join(x))

In [301]:
# taking a copy of final_dat in temp data
temp_data = final_data
temp_data

,id,title,weigh_avg_rating,details
0,278,The Shawshank Redemption,8.038342,drama crime timrobbins morganfreeman bobgunton...
1,550,Fight Club,7.921436,drama edwardnorton bradpitt meatloaf supportgr...
2,155,The Dark Knight,7.905627,drama action crime thriller christianbale heat...
3,680,Pulp Fiction,7.885403,thriller crime johntravolta samuell.jackson um...
4,27205,Inception,7.850750,action thriller sciencefiction mystery adventu...
...,...,...,...,...
4500,14164,Dragonball Evolution,5.451735,action adventure fantasy sciencefiction thrill...
4501,47933,Independence Day: Resurgence,5.407423,action adventure sciencefiction liamhemsworth ...
4502,241251,The Boy Next Door,5.382885,thriller jenniferlopez ryanguzman iannelson ma...
4503,415,Batman & Robin,5.273178,action crime fantasy georgeclooney chriso'donn...


# Vectorization

<H4> In this I am converting The details of column which is a Big String in the form of vectors and then compare those vectors similarity using cosine similarity and then recommend movies on the basis of similarity <br></H4>

In [302]:
# using tfidf vectorization and taking max_features = row size, and for ignoring stop words like a, is, am etc

tf = TfidfVectorizer(max_features = 4501, stop_words = 'english')

tfidf_matrix = tf.fit_transform(temp_data['details'])

In [303]:
tfidf_matrix.shape

(4501, 4501)

# Cosine Similarity 

In [304]:
# using sklearn library calculating cosine similarity

cosine_sim1 = linear_kernel(tfidf_matrix, tfidf_matrix)

In [305]:
# this will sort the movies acc to index in ascending order
temp_data = temp_data.reset_index()

# fetching movie titles
titles1 = temp_data['title']

# fetching movie titles coressponding to their indices
indices1 = pd.Series(temp_data.index, index = temp_data['title'])

In [306]:
# since 4797 will took time for interactive function to load
# hence i took some 50 movies

some_titles = titles1[1:51]

# or we can directly give title to the function

In [307]:
@interact
def recommendations(title = some_titles, frequency = range(3,11)):
    # getting index of movie
    idx = indices1[title]
    
    # getting the similarity score
    sim_scores = list(enumerate(cosine_sim1[idx]))
    
    # now sorting it
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # now storing top 10 movies we got according to similarity scores
    sim_scores = sim_scores[1:frequency+1]
    
    # storing the resulting movies indices
    movie_indices = [i[0] for i in sim_scores]
    
    return titles1.iloc[movie_indices]

interactive(children=(Dropdown(description='title', options=('Fight Club', 'The Dark Knight', 'Pulp Fiction', …

In [308]:
# here is one problem that it recommends similar movies according to genres, keywords, cast but it does not checking the ratings
# like batman and robin, superman returns are flop movies, it suggesting very similar but without thinking about the whether 
# it is bad or good according to rating
# but it still sugesting it. so I improved it using weighted_average_rating 

In [309]:
# so now I sort the movies not only on the basis on similarity but weighted_avgerage_rating also

In [310]:
# Here I use Count Vectorizer to find vectors corresponding to details string

word_vector = CountVectorizer(max_features = 4501, stop_words='english')
word_vector

CountVectorizer(max_features=4501, stop_words='english')

In [311]:
# creating a matrix of word_vector and converting it into 2-D Array

word_matrix = word_vector.fit_transform(temp_data['details']).toarray()
word_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [312]:
word_matrix.shape

(4501, 4501)

In [313]:
# Now Here I again Find the Cosine Similarity

cosine_sim2 = cosine_similarity(word_matrix)

In [314]:
# resetting the index
final_data = final_data.reset_index()

# fetching movie titles
titles2 = final_data['title']

# fetching indexes of movie titles
indices2 = pd.Series(final_data.index, index = final_data['title'])

In [315]:
@interact
def optimized_recommendations(title = some_titles, frequency = range(3,11)):
    
    # index of movie title
    idx = indices2[title]
    
    # finding similarity scores and sorting the resultant movies in descending order of sim_scores
    
    # enumerate function holds the indixes while calculating similarity
    sim_scores = list(enumerate(cosine_sim2[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    
    movie_indices = [i[0] for i in sim_scores]
    
    # here is the whole movie data required for applying average weighted technique 
    movie_req = final_data.iloc[movie_indices][['title', 'weigh_avg_rating']]
    
    
    result = movie_req.sort_values('weigh_avg_rating', ascending=False).head(frequency)
    
    return result

interactive(children=(Dropdown(description='title', options=('Fight Club', 'The Dark Knight', 'Pulp Fiction', …

In [316]:
# we can check the recommendations by typing titles
optimized_recommendations('X-Men Origins: Wolverine',10)

,title,weigh_avg_rating
63,Captain America: The Winter Soldier,7.216590
64,The Avengers,7.212929
68,Deadpool,7.201617
84,X-Men: Days of Future Past,7.153659
162,Captain America: Civil War,6.883206
198,Ant-Man,6.769996
329,X-Men,6.565666
352,X2,6.540143
919,The Wolverine,6.224674
943,X-Men: The Last Stand,6.217370


In [317]:
#pickle.dump(cosine_sim2,open('matrix.pkl','wb'))

In [318]:
@interact
def optimized_recommendations(title = some_titles, frequency = range(3,11)):
    
    # index of movie title
    idx = indices2[title]
    
    # finding similarity scores and sorting the resultant movies in descending order of sim_scores
    
    # enumerate function holds the indixes while calculating similarity
    sim_scores = list(enumerate(cosine_sim2[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    
    movie_indices = [i[0] for i in sim_scores]
    
    # here is the whole movie data required for applying average weighted technique 
    movie_req = final_data.iloc[movie_indices][['title', 'weigh_avg_rating']]
    
    return movie_req

interactive(children=(Dropdown(description='title', options=('Fight Club', 'The Dark Knight', 'Pulp Fiction', …